## <center>Intro to Python for Data Science with DC OpenData</center>

![dc flag](./images/4994-004-096A5339.jpg)

<small>This notebook was prepared by [Nicole Donnelly](mailto:nicole.donnelly@dc.gov) for the DC area regional Women in Data Science Conference [(DCMDVAWiDSRegional)](https://sites.google.com/view/dcmdvawidsregional/agenda?authuser=0) on March 5,2018 and presented as a one hour workshop.</small>

### Introduction
As cities embrace the [open data](https://en.wikipedia.org/wiki/Open_data) movement (you can find links to datasets for 85 cities [here](https://www.forbes.com/sites/metabrown/2017/06/30/quick-links-to-municipal-open-data-portals-for-85-us-cities/#290b91072290)), data scientist have an ever expanding population of data available to analyze and incorporate into other projects. As with any data source, unless you are designing and collecting it yourself, you will likely need to do some data wrangling before moving on to exploratory data analysis (EDA) and machine learning. 

During the course of this workshop, we will look at using [Python](https://www.python.org/) to wrangle [open data available from the Government of the District of Columbia](http://opendata.dc.gov/) in preparation for machine learning (this workshop will not cover machine learning). We will also look at some initial EDA via visualizations once we build a data set we want to use.

### Overview
If you do not have particular project in mind, I encourage you to [browse through the available data sets](http://opendata.dc.gov/datasets) (951 as of the time this workshop was created). We are going to start today with the [Computer Assisted Mass Appraisal - Condominium](http://opendata.dc.gov/datasets/computer-assisted-mass-appraisal-condominium) data. There is a lot that can be done with this data, particularly in conjunction with other data available from DC ([tax data](http://opendata.dc.gov/datasets/integrated-tax-system-public-extract), [crime data](http://opendata.dc.gov/datasets?q=crime), [construction data](http://opendata.dc.gov/datasets?q=construction), or [city service requests](http://opendata.dc.gov/datasets?q=311) for example) or other sources like the [United States Census Bureau](https://www.census.gov/data.html).

Buying a house in DC can be a daunting task. Inventory was being describe in November 2017 as ["dismally low"](https://www.washingtonpost.com/news/where-we-live/wp/2017/11/14/buyers-are-gaining-more-leverage-in-the-hot-d-c-area-housing-market/?utm_term=.b1aa57960214). But maybe armed with some appraisal data and machine learning, we can understand condominium values a little better. For example, maybe we could create a simple application to determine appraisal value, similar to [this example](https://github.com/georgetown-analytics/machine-learning/blob/master/examples/bbengfort/home%20sales/home_sales.ipynb) which uses housing sales data.

### Data

Here is [some information](https://www.arcgis.com/sharing/rest/content/items/d6c70978daa8461992658b69dccb3dbf/info/metadata/metadata.xml?format=default&output=html) available to us about the data.

**Abstract**: Computer Assisted Mass Appraisal (CAMA) database. The dataset contains attribution on housing characteristics for commercial properties, and was created as part of the DC Geographic Information System (DC GIS) for the D.C. Office of the Chief Technology Officer (OCTO) and participating D.C. government agencies. All DC GIS data is stored and exported in Maryland State Plane coordinates NAD 83 meters. 

METADATA CONTENT IS IN PROCESS OF VALIDATION AND SUBJECT TO CHANGE.

**Purpose**: This data is used for the planning and management of Washington, D.C. by local government agencies.

**Supplemental Information**: Most lots have one building in the cama file, assigned BLDG_NUM of one in the table. For parcels where multiple buildings exist, the primary building (such as the main residence) is assigned BLDG_NUM = 1. The other buildings or structures have BLDG_NUM values in random sequential order. After the primary structure, there is no way to associate BLDG_NUM > 2 records with any particular structure on the lot.



There is also some attribute information available. Some of it has been copied here. Not all of it is overly descriptive. 


***Entity and Attribute Information***:


**Attribute Label**: SALEDATE

**Attribute**:


**Attribute Label**: Sale_Num

**Attribute Definition**: sale number


**Attribute Label**: EYB

**Attribute Definition:** The calculated or apparent year, that an improvement was built that is most often more recent than actual year built.


**Attribute Label**: Shape

**Attribute Definition**: Feature geometry.


**Attribute Label**: OWNERNAME

**Attribute Definition**: property owner name


**Attribute Label**: SSL

**Attribute Definition**: square suffix and lot


**Attribute Label**: Extwall_D

**Attribute Definition**: exterior wall description


**Attribute Label**: PRICE

**Attribute**:


**Attribute Label**: Yr_Rmdl

**Attribute Definition**: year structure was remodeled


**Attribute Label**: Saledate

**Attribute Definition**: date of last sale


**Attribute Label**: AYB

**Attribute Definition**: The earliest time the main portion of the building was built. It is not affected by subsequent construction.


**Attribute Label**: Price

**Attribute Definition**: price of last sale


**Attribute Label**: GBA

**Attribute Definition**: gross building area in square feet


### Tools

This workshop has been created in [Jupyter Notebook](http://jupyter.org/) with [Python 3.6](https://www.python.org/downloads/release/python-360/). If you are unfamiliar with how to use a Jupyter notebook, consult [this tutorial](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/).

A popular package for working with data in python is [pandas](https://pandas.pydata.org/pandas-docs/stable/).

From the above link:

"**pandas** is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, **real world** data analysis in Python. Additionally, it has the broader goal of becoming **the most powerful and flexible open source data analysis / manipulation tool available in any language**. It is already well on its way toward this goal.

pandas is well suited for many different kinds of data:

* Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
* Ordered and unordered (not necessarily fixed-frequency) time series data.
* Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
* Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure"

We will also use [Seaborn](https://seaborn.pydata.org/) which is a visualization package built on [matplotlib](https://matplotlib.org/), a 2D plotting library in python. [openpyxl](https://pypi.python.org/pypi/openpyxl) is used to assist with writing files to an excel format for an optional part of the work below.

The following libraries, which are part of standard python, are also used:

* [os](https://docs.python.org/3/library/os.html)
* [urllib](https://docs.python.org/3/library/urllib.html)

After importing the packages and libraries needed, I am also setting two options that will assist in this exercise. `pd.options.display.max_columns = 35` is a pandas option that controls the number of columns displayed in a dataframe. Here we override the default and display 35. `%matplotlib inline` tells the notebook to display our plots in the notebook instead of in an external window.


In [ ]:
import os
import urllib
import openpyxl

import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
pd.options.display.max_columns = 35
%matplotlib inline

### Data Wrangling

Now we are ready to get our data. Storing the original files is always good practice so you can go back to the original data if needed. Below you will find a short python function to download data to a data directory. This way, if you decide to download more data later, you can set the information particular to that data as variables and re-use this function.

(There are a lot of ways data wrangling can be approached. Below is just one example.)

In [ ]:
# create a default data directory. Since we are using os, this convention will work on both Windows and
# *nix based environments
DATA_DIR = "./data"

# the two variables below are the url of our dataset on the opendata site and the path/name for the file
#we are downloading
cama_url = "https://opendata.arcgis.com/datasets/d6c70978daa8461992658b69dccb3dbf_24.csv"
cama_file = os.path.join(DATA_DIR, "cama-condo.csv")

In [ ]:
# the get_data function takes 3 variables - our data directory (dname), the url of our data (furl) and 
# the path/name for our file (fname)

def get_data(dname, furl, fname):
    
    # check to see if the data directory exists. if not, create it and print the message
    if not os.path.exists(dname):
        print("making directory")
        os.makedirs(dname)
    # if the data directory exists, just print the message
    else:
        print("directory exists")
        
    # check to see if the file exists. if not, download the file and print the message
    if not os.path.isfile(fname):
        print("downloading file")
        urllib.request.urlretrieve(furl, fname)
    
    # if the file exists, print the message (if you cloned the github repo, all the data is included)
    else:
        print("file exists")

Use the get_data function to download and save Computer Assisted Mass Appraisal - Condominium.

In [ ]:
get_data(DATA_DIR, cama_url, cama_file)

We are now going to use pandas to read the downloaded csv file into a dataframe called "df". There are a lot of options you can use when creating a dataframe. Take a look at the [documenation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [ ]:
df = pd.read_csv(cama_file)

We can use "head" to see the head, or first 5 rows of the data. If you want to see more than 5, put the number of rows you would like to see more rows, put the number you want inside the parenthesis.

In [ ]:
df.head()

In [ ]:
df.head(8)

Shape will tell us the shape -- number of rows and columns -- in our dataset. Columns will give us a list of column names.

In [ ]:
df.shape

In [ ]:
df.columns

Info will give us even more info. Take a few minutes to review the information.

In [ ]:
df.info()

Now that we have some data, let's start wrangling it in to something useful. If we use the idea mentioned above to create a simple application that uses machine learning to determine the appraisal value of a condominium in order to assist with the daunting task of purchasing in DC, we might want to start wrangling the dataset into something we can use to perform a [regression analysis](https://en.wikipedia.org/wiki/Regression_analysis). With this in mind, we will need data that contains numerical variables and we will also want to look at the correlations among these variables.

To add some additional information to help us work through the data, let's say our hypothetical buyer has decided they are only interested in condos with at least 2 bedrooms. Additionally, they would like to live in [Ward 6](https://planning.dc.gov/page/about-ward-6). The land area isn't really important to them.

Let's start by dropping some items we won't need. OBJECTID is a unique idea in the data and is not going to be useful in our regression. We don't have good information in our metadata on what QUALIFIED or USECODE are. We hadve decided LANDAREA is not important. GIS_LAST_MOD_DTTM is a modification date for the data, so also not relevant here. We use drop to drop columns in pandas. The axis variable of 1 specifies columns. "inplace" tells pandas we want to drop the columns from our actual dataframe. Without it, pandas will return a temporary dataframe object that excludes those columns. Again, the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html) will explain the options available for the command.

In [ ]:
df.drop(['OBJECTID', 'QUALIFIED', 'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM'], axis=1, inplace=True)

SALEDATE is probably something we want to hold on to. But as info showed us, the data is an object, not a date/time. Let's convert it to something more useable.

In [ ]:
df['SALEDATE'] = pd.to_datetime(df['SALEDATE'], errors='coerce')

Let's see what our data looks like now.

In [ ]:
df.info()

The number after the column name tells us the number of non-null values. It looks like we don't have a price for everything. PRICE has 48,934 non-null values while our data has 52,954. An object isn't going to be useful to us without a price and we don't really have an easy way to figure out what that missing price should be. So let's drop those items from the data.

In [ ]:
df = df[df.PRICE.notnull()]

In [ ]:
df.info()

We still have some missing values. Our hypothetical buyer wants to buy something with at least 2 bedrooms so next let's drop condos with less than 2 bedrooms from our data. We can do this by subsetting our dataframe to only items where the value in BEDRM >= 2. We then assign that to a variable we can call. By re-using our variable df, we are overwriting the data we are already referencing it with.

In [ ]:
df = df[df.BEDRM >= 2.0]

In [ ]:
df.info()

Next let's look at the column YR_RMDL. There are also a lot of null values and we have no meaningful way to impute them. Let's drop this column.

Our buyer isn't really concerned with the overall number of rooms as long as the condo has 2 bedrooms so we can drop that column as well. 

In [ ]:
df.drop(['YR_RMDL', 'ROOMS'], axis=1, inplace=True)

In [ ]:
df.info()

HF_BATHRM and FIREPLACES also contain null values. However, instead of dropping those columns, let's assume a null value is equivalent to 0. We can use fillna to change all our null values to 0. We can address HEAT and HEAT_D if we decide to use those in our regression.

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df.info()

Our buyer wants to live in Ward 6. How can we figure out which of the condos are located there? SSL (square suffix and lot) actually gives us a key to do that. We can determine the address from SSL. Once we have the addrss, we can get additional useful information, including Ward. 

Let's start with the address.

[Address Residential Units](http://opendata.dc.gov/datasets/address-residential-units): This table contains residential units and attributes of Address points, created as part of the Master Address Repository (MAR) for the D.C. Residential units can be condominiums or also apartments. Office of the Chief Technology Officer (OCTO) and DC Department of Consumer and Regulatory Affairs . It contains the addresses in the District of Columbia which are typically placed on the buildings. More information on the MAR can be found at http://dcgis.dc.gov.

We can download this data using our get_data function then read it into a dataframe, just like with our CAMA data.

In [ ]:
aru_url = "https://opendata.arcgis.com/datasets/c3c0ae91dca54c5d9ce56962fa0dd645_68.csv"
aru_file = os.path.join(DATA_DIR, "address_residential_unit.csv")

In [ ]:
get_data(DATA_DIR, aru_url, aru_file)

In [ ]:
aru_df = pd.read_csv(aru_file)

In [ ]:
aru_df.head()

In [ ]:
aru_df.shape

In [ ]:
aru_df.info()

SSL is common to both our datasets. Unfortunately, we seem to have a lot of null values in SSL with the address data. How many matches do we have between the two? There is a way to easily check that.

In [ ]:
df['SSL'].isin(aru_df['SSL']).value_counts()

pandas gives us a way to connect dataframes with merge. Here we are creating a third dataframe, condos, by merging our initial dataframe, df, with our address dataframe, aru_df. We also tell pandas that SSL is the common information between the two dataframes. By default, pandas performs an [inner join](https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins). You could also use a [left, right, or outer](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) join.

In [ ]:
condos = pd.merge(df, aru_df, on='SSL')

In [ ]:
condos.shape

In [ ]:
condos.head()

In [ ]:
condos.info()

Another useful thing we can look at is what values exist in our data, and how many times they occur in a particular column.


In [ ]:
print(condos.UNITTYPE.value_counts())
print('\n')
print(condos.STATUS.value_counts())

Our dataset didn't come with all that much information about it. Let's make the assumption we only want items that are ACTIVE. Then let's drop some columnds we don't need.

In [ ]:
condos = condos[condos.STATUS != 'RETIRE']

In [ ]:
condos.drop(['OBJECTID', 'STATUS', 'UNITTYPE', 'METADATA_ID'], axis=1, inplace=True)

In [ ]:
condos.info()

At this point we have addresses our units, but we don't know what Ward anything is in. The address dataset gave us an important bit of inforamation though. DC has a [Master Address Repository(MAR)](https://octo.dc.gov/node/1161947). And in the MAR [user guide](https://octo.dc.gov/sites/default/files/dc/sites/octo/publication/attachments/DCGIS-MARGeocoderUserGuide_1.pdf) we find out that we can get the Ward and a lot of other interesting things from the MAR. Unfortunately, the MAR application is only available for Windows 7 and 10.

In order to add the MAR information to our data, we can process our address via Access or Excel and run it as a batch. We currently have 19,563 condo units in our dataset. But since these are condo units, the number of unique address is lower so it makes sense to create a dataframe with our unique address, save those to and Excel file, and only process those address with the MAR application. I have done that separately and have included the resulting MAR encoded file in the github repo. I have included the steps here. If you would like to do this as well, you can change the following cell to a code cell and run it. We will need the mar_file variable later so run that bit of code regardless. 

In [ ]:
# create a variable with the path/name of the file that will contain your unique address list
mar_file = os.path.join(DATA_DIR, "addresses.xlsx")

```
# change this cell if you would like to perform the MAR encoding steps yourself

# create a writer object for the excel file
writer = pd.ExcelWriter(mar_file)

# create a dataframe that is just the unique addresses from our condos dataframe
addresses = pd.DataFrame(condos['FULLADDRESS'].unique(), columns=['full_address'])

# write the dataframe to excel and save the file
addresses.to_excel(writer, index=False)
writer.save()```

Information on the options for the MAR encoder are found in its documention. When I ran the unique address list, it took about 5 minutes. 

We can read the MAR encoded data back into a dataframe and join the data to our condos dataframe

In [ ]:
mar = pd.read_excel(mar_file)

In [ ]:
mar.info()

In [ ]:
condos = pd.merge(condos, mar, left_on='FULLADDRESS',  right_on='full_address')

In [ ]:
condos.shape

In [ ]:
condos.head()

In [ ]:
condos.MAR_WARD.value_counts()

Now we have the Ward data we need. Let's create another dataframe, condo_6, that contains the condos for Ward 6. Then drop the columns we won't need. Let's keep MAR_CENSUS_TRACT in case we want to use that later.

In [ ]:
condo_6 = condos[condos.MAR_WARD == 'Ward 6']

In [ ]:
condo_6.shape

In [ ]:
condo_6.drop(['full_address',  'MAR_MATCHADDRESS', 'MAR_XCOORD', 'MAR_YCOORD', 'MAR_LATITUDE', 'MAR_LONGITUDE', 
              'MAR_WARD', 'MAR_ZIPCODE', 'MARID', 'MAR_ERROR', 'MAR_SCORE', 'MAR_SOURCEOPERATION', 
              'MAR_IGNORE'], axis=1, inplace=True)

In [ ]:
condo_6.shape

In [ ]:
condo_6.info()

### Exploratory Data Analysis

Congratulations! You have just wrangled a dataset!

We set up a hypotheical scenario at the beginning of this workshop that guided our data wrangling. This is not the only thing that can be done with this data, nor is this the only way to perform data wrangling. Hopefully it was at least illustrative of things to thing about all the way.


All of those steps have lead us to a point where we can start doing some EDA and even machine learning on our data. Even though machine learning is outside the scope of this workshop, it is worth noting that if you are going to perform machine learning with something like [scikit-learn](http://scikit-learn.org/stable/) you will need to have numeric data. We do have a few columns with non-numeric data we would need to deal with before then, either by encoding the data to numeric values or dropping it from the data our models will use. 

We will leave those for now while we look at some ways to learn about our data.

Describe in pandas will provide us with some descriptive statistics about our data.

In [ ]:
condo_6.describe()

What do you learn about the data using describe?

It looks like in Ward 6, the average condo has 2.1 bedrooms, 1.8 bathrooms, has been sold twice, costs $460,416 dollars and has a living area of 1115.7 square feet. 

Seaborn gives us a lot of options to visualize the data. Let's look at the distribution of the sale price for the condos. By default, seaborn displays a histogram with the [kernel density estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation) (KDE) on top.

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
g = sns.distplot(condo_6.PRICE, rug=True, kde=True, ax=ax)
t = g.set_title("Distribution of Sale Prices")

It would also be interesting to understand the price based on the year the unit was last sold. We have the SALEDATE and can use that to group the sales prices by year in a [box plot](https://en.wikipedia.org/wiki/Box_plot).

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
g = sns.boxplot(y='PRICE', x=condo_6['SALEDATE'].dt.year, data=condo_6, ax=ax)
t = g.set_title("Distribution of Sale Price by Year")

Seaborn's joint plot allows you to view both a joint distribution and its marginals at once.

In [ ]:
g = sns.jointplot(y="PRICE", x="LIVING_GBA", data=condo_6, kind="hex", size=8)

Since we started with the idea of performing a regression with the data, we can also look at the correlations between our numeric variables in order to identify whether our not we have highly correlated variables. To do that, we will create (yet) another dataframe with the numerical variables. We can create a correlation matrix and provide that to the seaborn heatmap function to create a visual representation of correlation.

In [ ]:
numerical = condo_6[list(set(condo_6.columns) - set(['SSL', 'SALEDATE', 'FULLADDRESS', 'UNITNUM']))]
numerical.info()

In [ ]:
corr_matrix = numerical.corr()
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr_matrix, ax=ax);

I hope you found this introduction to python a useful starting point for getting started with open data for data science projects. 